In [1]:
import pandas as pd
import numpy as np
import pandas_datareader.data as rd
import datetime as dt
import statsmodels.api as st

In [2]:
#time horizon
end = dt.date(2021,3,22)
start = dt.date(end.year-5,end.month,end.day)
month = 12

In [3]:
funds = ["MSFT"]

In [4]:
data = pd.DataFrame()
data[funds] = rd.DataReader(funds, data_source="yahoo", start=start , end = end)["Adj Close"].pct_change()

In [5]:
data

,MSFT
Date,
2016-03-22,NaN
2016-03-23,-0.001849
2016-03-24,0.004447
2016-03-28,-0.012359
2016-03-29,0.021853
...,...
2021-03-16,0.012350
2021-03-17,-0.002819
2021-03-18,-0.026662


In [6]:
data_monthly = data.resample("M").agg(lambda x:(x+1).prod()-1)
data_monthly = data_monthly[1:]
data_monthly.head()

,MSFT
Date,
2016-04-30,-0.097049
2016-05-31,0.070197
2016-06-30,-0.034528
2016-07-31,0.107680
2016-08-31,0.020080


In [7]:
factors = rd.DataReader("F-F_Research_Data_Factors","famafrench",start,end)[0]

In [8]:
factors = factors[1:]
factors.head()

,Mkt-RF,SMB,HML,RF
Date,,,,
2016-04,0.92,0.69,3.28,0.01
2016-05,1.78,-0.27,-1.60,0.01
2016-06,-0.05,0.64,-1.40,0.02
2016-07,3.95,2.60,-1.23,0.02
2016-08,0.50,1.13,3.31,0.02


In [9]:
data_monthly.index = factors.index
data_monthly.head()

,MSFT
Date,
2016-04,-0.097049
2016-05,0.070197
2016-06,-0.034528
2016-07,0.107680
2016-08,0.020080


In [10]:
data_monthly.head()

,MSFT
Date,
2016-04,-0.097049
2016-05,0.070197
2016-06,-0.034528
2016-07,0.107680
2016-08,0.020080


In [11]:
merged_data = pd.merge(data_monthly,factors, on = "Date")
merged_data.head()

,MSFT,Mkt-RF,SMB,HML,RF
Date,,,,,
2016-04,-0.097049,0.92,0.69,3.28,0.01
2016-05,0.070197,1.78,-0.27,-1.60,0.01
2016-06,-0.034528,-0.05,0.64,-1.40,0.02
2016-07,0.107680,3.95,2.60,-1.23,0.02
2016-08,0.020080,0.50,1.13,3.31,0.02


In [12]:
merged_data[["Mkt-RF","SMB","HML","RF"]] = merged_data[["Mkt-RF","SMB","HML","RF"]]/100

In [13]:
merged_data.tail()

,MSFT,Mkt-RF,SMB,HML,RF
Date,,,,,
2020-11,0.060061,0.1247,0.0548,0.0211,0.0001
2020-12,0.039006,0.0463,0.0481,-0.0136,0.0001
2021-01,0.042892,-0.0003,0.0719,0.0285,0.0000
2021-02,0.004118,0.0278,0.0211,0.0708,0.0000
2021-03,0.015535,0.0309,-0.0248,0.0740,0.0000


R(i)-R(f) = β(a)*((R(M)-R(f))+β(b)*SMB +  β(c)*HML + α

In [14]:
merged_data["Real Return"] = merged_data.iloc[::,0]-merged_data.iloc[::,4]

In [15]:
merged_data.head()

,MSFT,Mkt-RF,SMB,HML,RF,Real Return
Date,,,,,,
2016-04,-0.097049,0.0092,0.0069,0.0328,0.0001,-0.097149
2016-05,0.070197,0.0178,-0.0027,-0.0160,0.0001,0.070097
2016-06,-0.034528,-0.0005,0.0064,-0.0140,0.0002,-0.034728
2016-07,0.107680,0.0395,0.0260,-0.0123,0.0002,0.107480
2016-08,0.020080,0.0050,0.0113,0.0331,0.0002,0.019880


In [16]:
y = merged_data["Real Return"]
X = merged_data[["Mkt-RF","SMB","HML"]]
x_indep = st.add_constant(X)

In [17]:
reg_model = st.OLS(y,x_indep)
results = reg_model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            Real Return   R-squared:                       0.542
Model:                            OLS   Adj. R-squared:                  0.517
Method:                 Least Squares   F-statistic:                     22.08
Date:                Sun, 23 May 2021   Prob (F-statistic):           1.46e-09
Time:                        11:57:51   Log-Likelihood:                 117.71
No. Observations:                  60   AIC:                            -227.4
Df Residuals:                      56   BIC:                            -219.0
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0132      0.005      2.715      0.009       0.003       0.023
Mkt-RF         0.8846      0.111      7.986      0.000       0.663       1.106
SMB           -0.3002      0.185     -1.627      0.109      -0.670       0.069
HML           -0.3663      0.134     -2.742      0.008      -0.634      -0.099
==============================================================================
Omnibus:                        3.239   Durbin-Watson:                   2.512
Prob(Omnibus):                  0.198   Jarque-Bera (JB):                2.586
Skew:                          -0.243   Prob(JB):                        0.274
Kurtosis:                       3.893   Cond. No.                         42.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""